In [0]:
pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-crpqjh83
  Stored in directory: /tmp/pip-ephem-wheel-cache-ucfxpsub/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [0]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, Dropout
from keras_vggface.vggface import VGGFace
from keras.utils import Sequence
import os
import wget
from zipfile import ZipFile

In [0]:
vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3), model='resnet50')

Instructions for updating:
Colocations handled automatically by placer.
94699520/94694792 [==============================] - 12s 0us/step


In [0]:
last_layer = vgg_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(2048, activation='relu', name='dense_relu')(x)
x = Dropout(0.1, seed=17)(x)
out = Dense(1, activation='sigmoid', name='classifier')(x)
custom_vgg_model = Model(vgg_model.input, out)

In [0]:
class FaceReplayAttackUQDataset(Sequence):  
  def __init__(self):
    if (not os.path.exists("data")):
      self.__download__()
    self.live_path = glob(os.path.join("data/live/.npy"))
    self.paper_path = glob(os.path.join("data/paper/*.npy"))
    

  def __download__(self):
    os.mkdir("data")
    wget.download("url", 'data') 
    with ZipFile('data/data.zip', 'r') as zipObj:
      zipObj.extractall("data")    
    

  def __len__(self):
    return max(len(self.live_path), len(self.paper_path))


  def __getitem__(self, idx):
    paper = np.read(self.paper_path[idx % len(self.paper_path)]) 
    live = np.read(self.live_path[idx % len(self.live_path)])
    X = np.concatenate((paper, live))
    y = np.concatenate((np.ones(len(paper)), np.zeros(len(live))))
    X, y = self.__unison_shuffled_copies(X, y)
    return X, y

<tf.Tensor 'avg_pool/AvgPool:0' shape=(?, 1, 1, 2048) dtype=float32>

In [0]:
if (not os.path.exists("checkpoints")):
  os.mkdir("checkpoints")
checkpoint = ModelCheckpoint("checkpoints", verbose=1, period=200)
callbacks_list = [checkpoint]

In [0]:
custom_vgg_model.fit_generator(generator, epochs=1000, verbose=1, callbacks=callbacks_list, shuffle=True)